In [1]:
'''
%pip install scikit-learn==1.1.2
%pip install scipy==1.9.1
%pip install missingpy==0.2.0
'''

'\n%pip install scikit-learn==1.1.2\n%pip install scipy==1.9.1\n%pip install missingpy==0.2.0\n'

In [2]:
import numpy as np
import pandas as pd
from data_split import k_fold_split
from forgetter import forget_random_col_per_sample

In [3]:
import sklearn.neighbors._base # this is from `scikit-learn` instead of `sklearn`
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest # remember to put this after sys.modules

In [4]:
qual_cols = { "age", "TSH", "T3", "TT4", "T4U", "FTI", "TBG" }
cat_cols = { "sex", "on_thyroxine", "on_antithyroid_meds", "sick", "pregnant", "thyroid_surgery", "I131_treatment", "lithium", "goitre", "tumor", "psych", "referral_source", "target" }

In [5]:
remove_qual_cols = { "TBG" }
remove_cat_cols = set()

In [6]:
sel_qual_cols = list(qual_cols.difference(remove_qual_cols))
sel_cat_cols = list(cat_cols.difference(remove_cat_cols))
display(sel_qual_cols)
display(sel_cat_cols)

['T4U', 'TSH', 'age', 'TT4', 'T3', 'FTI']

['target',
 'lithium',
 'sex',
 'thyroid_surgery',
 'pregnant',
 'I131_treatment',
 'goitre',
 'tumor',
 'sick',
 'psych',
 'on_antithyroid_meds',
 'referral_source',
 'on_thyroxine']

In [7]:
df = pd.read_csv("./data/thyroidDF.csv")
df

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,query_hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH_measured,TSH,T3_measured,T3,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
0,29,F,f,f,f,f,f,f,f,t,f,f,f,f,f,f,t,0.3,f,NaN,f,NaN,f,NaN,f,NaN,f,NaN,other,-,840801013
1,29,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,1.6,t,1.9,t,128.0,f,NaN,f,NaN,f,NaN,other,-,840801014
2,41,F,f,f,f,f,f,f,f,f,t,f,f,f,f,f,f,NaN,f,NaN,f,NaN,f,NaN,f,NaN,t,11.0,other,-,840801042
3,36,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,NaN,f,NaN,f,NaN,f,NaN,f,NaN,t,26.0,other,-,840803046
4,32,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,NaN,f,NaN,f,NaN,f,NaN,f,NaN,t,36.0,other,S,840803047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9167,56,M,f,f,f,f,f,f,f,f,t,f,f,f,f,f,f,NaN,f,NaN,t,64.0,t,0.83,t,77.0,f,NaN,SVI,-,870119022
9168,22,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,NaN,f,NaN,t,91.0,t,0.92,t,99.0,f,NaN,SVI,-,870119023
9169,69,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,NaN,f,NaN,t,113.0,t,1.27,t,89.0,f,NaN,SVI,I,870119025
9170,47,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f,NaN,f,NaN,t,75.0,t,0.85,t,88.0,f,NaN,other,-,870119027


In [8]:
cat_na_df = df[sel_cat_cols].dropna()
cat_na_df.shape

(8865, 13)

In [9]:
qual_na_df = df[sel_qual_cols].dropna()
qual_na_df.shape

(5984, 6)

In [10]:
df.dropna().shape

(20, 31)

In [11]:
weight_qual_map = {
    "age" : 1,
    "T3"  : 1,
    "T4U" : 1,
    "TSH" : 1,
    "TT4" : 1,
    "FTI" : 1
}

In [12]:
weight_cat_map = {
    "sick" : 1,
    "referral_source" : 1,
    "on_antithyroid_meds" : 1,
    "lithium" : 1,
    "goitre" : 1,
    "tumor" : 1,
    "thyroid_surgery" : 1,
    "sex" : 1,
    "target" : 1,
    "on_thyroxine" : 1,
    "psych" : 1,
    "I131_treatment" : 1,
    "pregnant" : 1
}

In [13]:
k = 5

In [14]:
print(f"CAT:\ntrain: {cat_na_df.shape[0] / k * (k-1)}\ntest: {cat_na_df.shape[0] / k}")

CAT:
train: 7092.0
test: 1773.0


In [15]:
print(f"QUAL:\ntrain: {qual_na_df.shape[0] / k * (k-1)}\ntest: {qual_na_df.shape[0] / k}")

QUAL:
train: 4787.2
test: 1196.8


In [16]:
train_cat_na_df, test_cat_na_df = k_fold_split(cat_na_df, k)

In [17]:
train_qual_na_df, test_qual_na_df = k_fold_split(qual_na_df, k)

/workspaces/imputation-methods/data_split.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  folds[fold_idx] = pd.concat([folds[fold_idx], curr_fold.iloc[fold_idx].to_frame().T]) if curr_fold.shape[
/workspaces/imputation-methods/data_split.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  folds[fold_idx] = pd.concat([folds[fold_idx], curr_fold.iloc[fold_idx].to_frame().T]) if curr_fold.shape[
/workspaces/imputation-methods/data_split.py:34: FutureWarning: The behavior of DataFrame concaten

In [18]:
seed = 11
random_generator = np.random.default_rng(seed)

In [39]:
train_cat_na_missing_vals_df, train_missing_col_map, train_cat_missing_vals_idxs = forget_random_col_per_sample(train_cat_na_df, weight_cat_map, random_generator)
test_cat_na_missing_vals_df, test_missing_col_map, test_cat_missing_vals_idxs = forget_random_col_per_sample(test_cat_na_df, weight_cat_map, random_generator)

In [35]:
#imputer = MissForest(missing_values=np.NaN)
#X_cat = train_cat_na_missing_vals_df.to_numpy()
#X_cat_imputed = imputer.fit_transform(X_cat, cat_vars=list(range(X_cat.shape[1])))

In [40]:
train_qual_na_missing_vals_df, train_missing_col_map, train_qual_missing_vals_idxs = forget_random_col_per_sample(train_qual_na_df, weight_qual_map, random_generator)
test_qual_na_missing_vals_df, test_missing_col_map, test_qual_missing_vals_idxs = forget_random_col_per_sample(test_qual_na_df, weight_qual_map, random_generator)

In [34]:
imputer = MissForest(missing_values=np.NaN)
X_qual = train_qual_na_missing_vals_df.to_numpy()
X_qual_imputed = imputer.fit_transform(X_qual)

/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been depr

Iteration: 0


/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been depr

Iteration: 1


/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been depr

Iteration: 2


In [38]:
imputer.statistics_

{'col_means': array([  0.97311553,   5.60820178,  86.38128951, 107.86764068,
          1.9669808 , 112.93893633]),
 'col_modes': None}

In [41]:
imputed_qual_df = pd.DataFrame(data=X_qual_imputed, index=train_qual_missing_vals_idxs, columns=sel_qual_cols)

In [42]:
imputed_qual_df

,T4U,TSH,age,TT4,T3,FTI
519,1.7319,0.400,21.00,162.0,3.700,92.00
8943,1.1200,4.800,50.56,164.0,2.400,146.00
3744,0.6900,0.035,57.67,87.0,0.300,126.00
430,1.0172,213.000,65.00,43.0,0.900,45.00
5435,0.8100,1.800,53.00,104.0,1.752,128.00
...,...,...,...,...,...,...
1480,0.9400,0.300,35.00,125.0,2.000,132.98
3001,1.0500,5.100,37.00,84.0,2.500,80.27
172,1.0427,0.300,63.00,130.0,1.300,124.00
4851,0.9000,0.045,50.05,75.0,1.500,83.00
